In [10]:
import pandas as pd
import numpy as np
from PIL import Image
from IPython.display import display

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 1. Load the CSV file
file_name = 'mnist_test.csv'
df = pd.read_csv(file_name)

# 2. Separate labels and pixel values
labels = df['label'].values
pixels = df.drop('label', axis=1).values.astype(np.uint8)

# 3. Ask the user to enter an index
max_idx = len(labels) - 1
idx = int(input(f"Enter image index (0 to {max_idx}): "))

# Validate the index
if not (0 <= idx <= max_idx):
    raise ValueError(f"Index out of range. Must be between 0 and {max_idx}.")

# 4. Reshape pixel data and display image
img_array = pixels[idx].reshape(28, 28)
img = Image.fromarray(img_array, mode='L')
display(img)

# 5. Print the label
print("Label:", labels[idx])
image = pixels[idx].reshape(1, 784)
print(image[0])

class Layer:
    _slots_ = ('_number', '_size', '_data')

    def _init_(self, number: int, size: int):
        self._number = number
        self._size = size
        self._data = np.zeros((1, size), dtype=np.float32)

    @property
    def number(self) -> int:
        return self._number

    @property
    def size(self) -> int:
        return self._size

    @property
    def data(self) -> np.ndarray:
        return self._data

    @data.setter
    def data(self, new_data: np.ndarray):
        if new_data.shape != self._data.shape:
            raise ValueError(f"Expected shape {self._data.shape}, got {new_data.shape}")
        self._data = new_data


class Weight:
    _slots_ = ('_value',)

    def _init_(self, from_layer: Layer, to_layer: Layer):
        self._value = np.random.randn(from_layer.size, to_layer.size).astype(np.float32)

    @property
    def value(self) -> np.ndarray:
        return self._value

    @value.setter
    def value(self, new_value: np.ndarray):
        if new_value.shape != self._value.shape:
            raise ValueError(f"Expected shape {self._value.shape}, got {new_value.shape}")
        self._value = new_value


class Bias:
    _slots_ = ('_value',)

    def _init_(self, to_layer: Layer):
        self._value = np.random.randn(1, to_layer.size).astype(np.float32)

    @property
    def value(self) -> np.ndarray:
        return self._value

    @value.setter
    def value(self, new_value: np.ndarray):
        if new_value.shape != self._value.shape:
            raise ValueError(f"Expected shape {self._value.shape}, got {new_value.shape}")
        self._value = new_value

import numpy as np

def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))


class Neural:
    _slots_ = ('_activation', '_layers', '_weights', '_biases')

    def _init_(self, activation=sigmoid):
        self._activation = activation
        self._layers = []
        self._weights = []
        self._biases = []

    def add_layer(self, layer):
        self._layers.append(layer)
        if len(self._layers) > 1:
            self._weights.append(Weight(self._layers[-2], self._layers[-1]))
            self._biases.append(Bias(self._layers[-1]))

    def forward(self):
        for i in range(1, len(self._layers)):
            prev_data = self._layers[i - 1].data
            weights = self._weights[i - 1].value
            bias = self._biases[i - 1].value
            z = prev_data @ weights + bias  # Faster and more readable
            self._layers[i].data = self._activation(z)

    @property
    def output(self):
        return self._layers[-1].data if self._layers else None

# Assuming image is a NumPy array of shape (1, 784) — a flattened MNIST image
net = Neural(sigmoid)

# Add input layer
input_layer = Layer(1, 784)
input_layer.data = image.astype(np.float32) / 255.0
net.add_layer(input_layer)

# Add hidden and output layers
net.add_layer(Layer(2, 107))  # Hidden Layer 1
net.add_layer(Layer(3, 26))   # Hidden Layer 2
net.add_layer(Layer(4, 10))   # Output Layer (10 digits)

# Forward pass
net.forward()

# Prediction
predicted_digit = np.argmax(net.output)
print("Predicted Digit:", predicted_digit)

FileNotFoundError: [Errno 2] No such file or directory: 'mnist_test.csv'

In [9]:
import tkinter as tk
from tkinter import messagebox, ttk
import webbrowser
import os
import json

HISTORY_FILE = "url_history.json"

# Load last URL from history
def load_last_url():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, 'r') as f:
            data = json.load(f)
            return data.get("last_url", "")
    return ""

# Save URL to history
def save_url(url):
    with open(HISTORY_FILE, 'w') as f:
        json.dump({"last_url": url}, f)

# Validate and fix URL
def validate_url(url):
    url = url.strip()
    if not url.startswith("http://") and not url.startswith("https://"):
        url = "https://" + url
    return url

# Open the URL
def open_url():
    url = validate_url(url_entry.get())
    mode = mode_var.get()

    if not url or url.isspace():
        messagebox.showerror("Error", "Please enter a valid URL.")
        return

    save_url(url)

    try:
        if mode == "tab":
            webbrowser.open_new_tab(url)
        elif mode == "window":
            webbrowser.open_new(url)
        else:
            messagebox.showwarning("Warning", "Please select how to open the URL.")
            return
        messagebox.showinfo("Opened", f"Successfully opened:\n{url}")
    except Exception as e:
        messagebox.showerror("Failed", f"Could not open URL:\n{e}")

# Toggle dark mode
def toggle_dark_mode():
    global dark_mode
    dark_mode = not dark_mode
    bg = "#222" if dark_mode else "#f0f0f0"
    fg = "#fff" if dark_mode else "#000"

    root.configure(bg=bg)
    for widget in root.winfo_children():
        widget.configure(bg=bg, fg=fg)
    style.configure('TButton', background=bg, foreground=fg)
    style.configure('TRadiobutton', background=bg, foreground=fg)
    dark_button.configure(text="☀ Light Mode" if dark_mode else "🌙 Dark Mode")

# Main GUI
root = tk.Tk()
root.title("🌐 Open Website")
root.geometry("420x280")
style = ttk.Style()
dark_mode = False

# Widgets
url_label = tk.Label(root, text="Enter Website URL:", font=('Arial', 12))
url_label.pack(pady=5)

url_entry = tk.Entry(root, width=50)
url_entry.pack(pady=5)
url_entry.insert(0, load_last_url())  # Load previous URL

mode_var = tk.StringVar()
mode_label = tk.Label(root, text="Open in:", font=('Arial', 11))
mode_label.pack(pady=5)

tab_radio = tk.Radiobutton(root, text="New Tab", variable=mode_var, value="tab")
tab_radio.pack()
window_radio = tk.Radiobutton(root, text="New Window", variable=mode_var, value="window")
window_radio.pack()

open_button = tk.Button(root, text="Open URL", command=open_url, bg="green", fg="white", font=('Arial', 12))
open_button.pack(pady=10)

dark_button = tk.Button(root, text="🌙 Dark Mode", command=toggle_dark_mode, font=('Arial', 10))
dark_button.pack()

root.mainloop()

In [15]:
import openai
import smtplib
import os
import json
from tkinter import *
from tkinter import filedialog, messagebox, ttk
from email.message import EmailMessage
from datetime import datetime

# === GLOBALS ===
HISTORY_FILE = "email_history.json"
LAST_MESSAGE_FILE = "last_email.txt"
DARK_MODE = True

BG_COLOR = "#2c3e50" if DARK_MODE else "#f0f4f8"
FG_COLOR = "#ecf0f1" if DARK_MODE else "#2c3e50"
ENTRY_BG = "#34495e" if DARK_MODE else "white"
BTN_COLOR = "#27ae60" if DARK_MODE else "#2980b9"
def generate_gpt_message(name, topic, tone, api_key):
    openai.api_key = api_key
    prompt = (
        f"Write a {tone.lower()} email to {name} about '{topic}'. "
        "Sign it as 'Your Automated Assistant'."
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful email assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=300
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error: {e}"
def send_email():
    name = name_entry.get()
    to_email = to_email_entry.get()
    topic = topic_entry.get()
    from_email = your_email_entry.get()
    app_password = app_pass_entry.get()
    openai_key = openai_key_entry.get()
    tone = tone_var.get()
    subject = f"Regarding {topic.title()}"
    attachment_path = attachment_label['text']

    if not all([name, to_email, topic, from_email, app_password, openai_key]):
        messagebox.showerror("Error", "All fields except attachment are required.")
        return

    body = generate_gpt_message(name, topic, tone, openai_key)
    if body.startswith("Error:"):
        messagebox.showerror("GPT Error", body)
        return

    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    msg.set_content(body)

    if attachment_path and os.path.isfile(attachment_path):
        try:
            with open(attachment_path, 'rb') as f:
                file_data = f.read()
                file_name = os.path.basename(attachment_path)
                msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)
        except Exception as e:
            messagebox.showerror("Attachment Error", f"Failed to attach file: {e}")
            return

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(from_email, app_password)
            smtp.send_message(msg)

        save_to_history(name, to_email, topic, tone, body)
        export_last_email(subject, body)

        messagebox.showinfo("Success", "✅ Email sent successfully!")

    except Exception as e:
        messagebox.showerror("SMTP Error", f"❌ Failed to send email: {e}")

def save_to_history(name, to_email, topic, tone, body):
    record = {
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "name": name,
        "to_email": to_email,
        "topic": topic,
        "tone": tone,
        "message": body
    }

    history = []
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, "r") as f:
            history = json.load(f)
    history.append(record)

    with open(HISTORY_FILE, "w") as f:
        json.dump(history, f, indent=2)

def export_last_email(subject, message):
    with open(LAST_MESSAGE_FILE, "w") as f:
        f.write(f"Subject: {subject}\n\n{message}")

def choose_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        attachment_label.config(text=file_path)
root = Tk()
root.title("Email Sending")
root.geometry("650x600")
root.configure(bg=BG_COLOR)

Label(root, text=" Email Sender ", font=("Helvetica", 20, "bold"), bg=BG_COLOR, fg=FG_COLOR).pack(pady=15)

def field(label_text, var=None, is_pass=False):
    frame = Frame(root, bg=BG_COLOR)
    frame.pack(pady=5)
    Label(frame, text=label_text, font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
    entry = Entry(frame, width=50, font=("Helvetica", 11), bg=ENTRY_BG, fg=FG_COLOR, show="*" if is_pass else "")
    entry.pack()
    return entry

name_entry = field("Recipient Name:")
to_email_entry = field("Recipient Email:")
topic_entry = field("Email Topic:")
your_email_entry = field("Your Gmail:")
app_pass_entry = field("App Password:", is_pass=True)
openai_key_entry = field("OpenAI API Key:", is_pass=True)

# Tone Selector
frame = Frame(root, bg=BG_COLOR)
frame.pack(pady=5)
Label(frame, text="Email Tone:", font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
tone_var = StringVar(value="friendly")
ttk.Combobox(frame, textvariable=tone_var, values=["friendly", "formal", "flirty", "funny", "strict"], width=47).pack()

# File chooser
Button(root, text="📎 Choose Attachment", command=choose_file, bg="#2980b9", fg="white", font=("Helvetica", 10)).pack(pady=8)
attachment_label = Label(root, text="", bg=BG_COLOR, fg="#bdc3c7", font=("Helvetica", 9))
attachment_label.pack()

# Send button
Button(root, text="🚀 Send Email", command=send_email, bg=BTN_COLOR, fg="white", font=("Helvetica", 13, "bold"), width=30).pack(pady=20)

root.mainloop()

In [14]:
import pandas as pd
import numpy as np
from PIL import Image
from IPython.display import display

# Activation functions
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

def softmax(x: np.ndarray) -> np.ndarray:
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=1, keepdims=True)

# Load dataset
file_name = r"C:\Users\Rohit\Downloads\archive"  # Ensure this file exists in your working directory
df = pd.read_csv(file_name)

# Separate labels and pixels
labels = df['label'].values
pixels = df.drop('label', axis=1).values.astype(np.uint8)

# Get user input
max_idx = len(labels) - 1
idx = int(input(f"Enter image index (0 to {max_idx}): "))

if not (0 <= idx <= max_idx):
    raise ValueError(f"Index out of range. Must be between 0 and {max_idx}.")

# Show image
img_array = pixels[idx].reshape(28, 28)
img = Image.fromarray(img_array, mode='L')
display(img)
print("Label:", labels[idx])
image = pixels[idx].reshape(1, 784)
print("Flattened Image Data:", image[0])

# Neural network components
class Layer:
    def __init__(self, number: int, size: int):
        self._number = number
        self._size = size
        self._data = np.zeros((1, size), dtype=np.float32)

    @property
    def number(self) -> int:
        return self._number

    @property
    def size(self) -> int:
        return self._size

    @property
    def data(self) -> np.ndarray:
        return self._data

    @data.setter
    def data(self, new_data: np.ndarray):
        if new_data.shape != self._data.shape:
            raise ValueError(f"Expected shape {self._data.shape}, got {new_data.shape}")
        self._data = new_data

class Weight:
    def __init__(self, from_layer: Layer, to_layer: Layer):
        self._value = np.random.randn(from_layer.size, to_layer.size).astype(np.float32)

    @property
    def value(self) -> np.ndarray:
        return self._value

    @value.setter
    def value(self, new_value: np.ndarray):
        if new_value.shape != self._value.shape:
            raise ValueError(f"Expected shape {self._value.shape}, got {new_value.shape}")
        self._value = new_value

class Bias:
    def __init__(self, to_layer: Layer):
        self._value = np.random.randn(1, to_layer.size).astype(np.float32)

    @property
    def value(self) -> np.ndarray:
        return self._value

    @value.setter
    def value(self, new_value: np.ndarray):
        if new_value.shape != self._value.shape:
            raise ValueError(f"Expected shape {self._value.shape}, got {new_value.shape}")
        self._value = new_value

class Neural:
    def __init__(self, activation=sigmoid):
        self._activation = activation
        self._layers = []
        self._weights = []
        self._biases = []

    def add_layer(self, layer):
        self._layers.append(layer)
        if len(self._layers) > 1:
            self._weights.append(Weight(self._layers[-2], self._layers[-1]))
            self._biases.append(Bias(self._layers[-1]))

    def forward(self):
        for i in range(1, len(self._layers)):
            prev_data = self._layers[i - 1].data
            weights = self._weights[i - 1].value
            bias = self._biases[i - 1].value
            z = prev_data @ weights + bias
            self._layers[i].data = self._activation(z)

    @property
    def output(self):
        return self._layers[-1].data if self._layers else None

# Create neural network
net = Neural(sigmoid)

# Input Layer
input_layer = Layer(1, 784)
input_layer.data = image.astype(np.float32) / 255.0  # Normalize input
net.add_layer(input_layer)

# Hidden and Output Layers
net.add_layer(Layer(2, 107))  # Hidden layer 1
net.add_layer(Layer(3, 26))   # Hidden layer 2
net.add_layer(Layer(4, 10))   # Output layer (10 classes)

# Forward pass
net.forward()

# Output and prediction
output = net.output
predicted_digit = np.argmax(output)
print("Predicted Digit:", predicted_digit)

# Apply softmax to get probabilities
probs = softmax(output)

# Cross-entropy loss calculation
true_label = labels[idx]
loss = -np.log(probs[0, true_label])
print("Cross-Entropy Loss:", loss)


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Rohit\\Downloads\\archive'

In [17]:

import os
import json
import smtplib
from tkinter import *
from tkinter import filedialog, messagebox, ttk
from email.message import EmailMessage
from datetime import datetime
import google.generativeai as genai

# === CONFIG ===
HISTORY_FILE = "email_history.json"
LAST_EMAIL_FILE = "last_email.txt"
DARK_MODE = False

BG_COLOR = "#2c3e50" if DARK_MODE else "#f0f4f8"
FG_COLOR = "#ecf0f1" if DARK_MODE else "#2c3e50"
ENTRY_BG = "#34495e" if DARK_MODE else "white"
BTN_COLOR = "#27ae60"

# === Gemini Setup ===
def setup_gemini(api_key):
    genai.configure(api_key=api_key)

def generate_gemini_message(name, topic, tone):
    prompt = f"Write a {tone.lower()} email to {name} about '{topic}'. Sign it as 'Your Automated Assistant'."
    try:
        model = genai.GenerativeModel('gemini-pro')
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Error: {e}"

# === Email Sending ===
def send_email():
    name = name_entry.get()
    to_email = to_email_entry.get()
    topic = topic_entry.get()
    from_email = your_email_entry.get()
    app_password = app_pass_entry.get()
    gemini_key = gemini_key_entry.get()
    tone = tone_var.get()
    subject = f"Regarding {topic.title()}"
    attachment_path = attachment_label['text']

    if not all([name, to_email, topic, from_email, app_password, gemini_key]):
        messagebox.showerror("Error", "All fields except attachment are required.")
        return

    setup_gemini(gemini_key)
    body = generate_gemini_message(name, topic, tone)

    if body.startswith("Error:"):
        messagebox.showerror("Gemini Error", body)
        return

    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    msg.set_content(body)

    if attachment_path and os.path.isfile(attachment_path):
        try:
            with open(attachment_path, 'rb') as f:
                file_data = f.read()
                file_name = os.path.basename(attachment_path)
                msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)
        except Exception as e:
            messagebox.showerror("Attachment Error", f"Failed to attach file: {e}")
            return

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(from_email, app_password)
            smtp.send_message(msg)

        save_to_history(name, to_email, topic, tone, body)
        export_last_email(subject, body)
        messagebox.showinfo("Success", "✅ Email sent successfully!")
    except Exception as e:
        messagebox.showerror("SMTP Error", f"❌ Failed to send email: {e}")

def save_to_history(name, to_email, topic, tone, message):
    record = {
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "to": to_email,
        "name": name,
        "topic": topic,
        "tone": tone,
        "message": message
    }
    history = []
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, "r") as f:
            history = json.load(f)
    history.append(record)
    with open(HISTORY_FILE, "w") as f:
        json.dump(history, f, indent=2)

def export_last_email(subject, body):
    with open(LAST_EMAIL_FILE, "w") as f:
        f.write(f"Subject: {subject}\n\n{body}")

def choose_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        attachment_label.config(text=file_path)

# === GUI ===
root = Tk()
root.title("📧 Gemini Email Sender")
root.geometry("650x600")
root.configure(bg=BG_COLOR)

Label(root, text="Gemini Email Sender", font=("Helvetica", 20, "bold"), bg=BG_COLOR, fg=FG_COLOR).pack(pady=15)

def field(label_text, is_pass=False):
    frame = Frame(root, bg=BG_COLOR)
    frame.pack(pady=5)
    Label(frame, text=label_text, font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
    entry = Entry(frame, width=50, font=("Helvetica", 11), bg=ENTRY_BG, fg=FG_COLOR, show="*" if is_pass else "")
    entry.pack()
    return entry

name_entry = field("Recipient Name:")
to_email_entry = field("Recipient Email:")
topic_entry = field("Email Topic:")
your_email_entry = field("Your Gmail:")
app_pass_entry = field("Gmail App Password:", is_pass=True)
gemini_key_entry = field("Gemini API Key:", is_pass=True)

# Tone dropdown
tone_var = StringVar(value="friendly")
tone_frame = Frame(root, bg=BG_COLOR)
tone_frame.pack(pady=5)
Label(tone_frame, text="Select Tone:", font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
ttk.Combobox(tone_frame, textvariable=tone_var, values=["friendly", "formal", "flirty", "funny", "strict"], width=47).pack()

# Attachment chooser
Button(root, text="📎 Choose Attachment", command=choose_file, bg="#3498db", fg="white", font=("Helvetica", 10)).pack(pady=5)
attachment_label = Label(root, text="", bg=BG_COLOR, fg="#bdc3c7", font=("Helvetica", 9))
attachment_label.pack()

# Send button
Button(root, text="🚀 Send Email", command=send_email, bg=BTN_COLOR, fg="white", font=("Helvetica", 13, "bold"), width=30).pack(pady=20)

root.mainloop()

C:\Users\Rohit\PycharmProjects\PythonProject2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
import json
import smtplib
from tkinter import *
from tkinter import filedialog, messagebox, ttk
from email.message import EmailMessage
from datetime import datetime
import google.generativeai as genai

# === CONFIG ===
HISTORY_FILE = "email_history.json"
LAST_EMAIL_FILE = "last_email.txt"
DARK_MODE = True

BG_COLOR = "#669BBC" if DARK_MODE else "#f0f4f8"
FG_COLOR = "#ecf0f1" if DARK_MODE else "#2c3e50"
ENTRY_BG = "#34495e" if DARK_MODE else "white"
BTN_COLOR = "#780000"

# === Gemini Setup ===
def setup_gemini(api_key):
    genai.configure(api_key=api_key)

def generate_gemini_message(name, topic, tone):
    prompt = f"Write a {tone.lower()} email to {name} about '{topic}'. Sign it as 'Your Automated Assistant'."
    try:
        model = genai.GenerativeModel(model_name="gemini-pro")
        response = model.generate_content([prompt])  # <-- Important fix
        return response.text.strip()
    except Exception as e:
        return f"Error: {e}"

# === Email Sending ===
def send_email():
    name = name_entry.get()
    to_email = to_email_entry.get()
    topic = topic_entry.get()
    from_email = your_email_entry.get()
    app_password = app_pass_entry.get()
    gemini_key = gemini_key_entry.get()
    tone = tone_var.get()
    subject = f"Regarding {topic.title()}"
    attachment_path = attachment_label['text']

    if not all([name, to_email, topic, from_email, app_password, gemini_key]):
        messagebox.showerror("Error", "All fields except attachment are required.")
        return

    setup_gemini(gemini_key)
    body = generate_gemini_message(name, topic, tone)

    if body.startswith("Error:"):
        messagebox.showerror("Gemini Error", body)
        return

    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email
    msg.set_content(body)

    if attachment_path and os.path.isfile(attachment_path):
        try:
            with open(attachment_path, 'rb') as f:
                file_data = f.read()
                file_name = os.path.basename(attachment_path)
                msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)
        except Exception as e:
            messagebox.showerror("Attachment Error", f"Failed to attach file: {e}")
            return

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(from_email, app_password)
            smtp.send_message(msg)

        save_to_history(name, to_email, topic, tone, body)
        export_last_email(subject, body)
        messagebox.showinfo("Success", "✅ Email sent successfully!")
    except Exception as e:
        messagebox.showerror("SMTP Error", f"❌ Failed to send email: {e}")

def save_to_history(name, to_email, topic, tone, message):
    record = {
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "to": to_email,
        "name": name,
        "topic": topic,
        "tone": tone,
        "message": message
    }
    history = []
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, "r") as f:
            history = json.load(f)
    history.append(record)
    with open(HISTORY_FILE, "w") as f:
        json.dump(history, f, indent=2)

def export_last_email(subject, body):
    with open(LAST_EMAIL_FILE, "w") as f:
        f.write(f"Subject: {subject}\n\n{body}")

def choose_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        attachment_label.config(text=file_path)

# === GUI ===
root = Tk()
root.title("Email Sender")
root.geometry("650x600")
root.configure(bg=BG_COLOR)

Label(root, text="Email Sender", font=("Helvetica", 20, "bold"), bg=BG_COLOR, fg=FG_COLOR).pack(pady=15)

def field(label_text, is_pass=False):
    frame = Frame(root, bg=BG_COLOR)
    frame.pack(pady=5)
    Label(frame, text=label_text, font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
    entry = Entry(frame, width=50, font=("Helvetica", 11), bg=ENTRY_BG, fg=FG_COLOR, show="*" if is_pass else "")
    entry.pack()
    return entry

name_entry = field("Recipient Name:")
to_email_entry = field("Recipient Email:")
topic_entry = field("Email Topic:")
your_email_entry = field("Your Gmail:")
app_pass_entry = field("Gmail Password:", is_pass=True)
gemini_key_entry = field("API Key:", is_pass=True)

# Tone dropdown
tone_var = StringVar(value="friendly")
tone_frame = Frame(root, bg=BG_COLOR)
tone_frame.pack(pady=5)
Label(tone_frame, text="Select Tone:", font=("Helvetica", 12), bg=BG_COLOR, fg=FG_COLOR).pack(anchor="w")
ttk.Combobox(tone_frame, textvariable=tone_var, values=["friendly", "formal", "flirty", "funny", "strict"], width=47).pack()

# Attachment chooser
Button(root, text="📎 Choose Attachment", command=choose_file, bg="#3498db", fg="white", font=("Helvetica", 10)).pack(pady=5)
attachment_label = Label(root, text="", bg=BG_COLOR, fg="#bdc3c7", font=("Helvetica", 9))
attachment_label.pack()

# Send button
Button(root, text="🚀 Send Email", command=send_email, bg=BTN_COLOR, fg="white", font=("Helvetica", 13, "bold"), width=30).pack(pady=20)

root.mainloop()

In [12]:
import tkinter as tk
from tkinter import messagebox, ttk
import webbrowser
import os
import json

HISTORY_FILE = "url_history.json"


# Load last URL from history
def load_last_url():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, 'r') as f:
            data = json.load(f)
            return data.get("last_url", "")
    return ""


# Save URL to history
def save_url(url):
    with open(HISTORY_FILE, 'w') as f:
        json.dump({"last_url": url}, f)


# Validate and fix URL
def validate_url(url):
    url = url.strip()
    if not url.startswith("http://") and not url.startswith("https://"):
        url = "https://" + url
    return url


# Open the URL
def open_url():
    url = validate_url(url_entry.get())
    mode = mode_var.get()

    if not url or url.isspace():
        messagebox.showerror("Error", "Please enter a valid URL.")
        return

    save_url(url)

    try:
        if mode == "tab":
            webbrowser.open_new_tab(url)
        elif mode == "window":
            webbrowser.open_new(url)
        else:
            messagebox.showwarning("Warning", "Please select how to open the URL.")
            return
        messagebox.showinfo("Opened", f"Successfully opened:\n{url}")
    except Exception as e:
        messagebox.showerror("Failed", f"Could not open URL:\n{e}")


# Toggle dark mode
def toggle_dark_mode():
    global dark_mode
    dark_mode = not dark_mode
    bg = "#222" if dark_mode else "#f0f0f0"
    fg = "#fff" if dark_mode else "#000"

    root.configure(bg=bg)
    for widget in root.winfo_children():
        widget.configure(bg=bg, fg=fg)
    style.configure('TButton', background=bg, foreground=fg)
    style.configure('TRadiobutton', background=bg, foreground=fg)
    dark_button.configure(text="☀ Light Mode" if dark_mode else "🌙 Dark Mode")


# Main GUI
root = tk.Tk()
root.title("Open Website")
root.geometry("420x280")
style = ttk.Style()
dark_mode = False

# Widgets
url_label = tk.Label(root, text="Enter Website URL:", font=('Arial', 12))
url_label.pack(pady=5)

url_entry = tk.Entry(root, width=50)
url_entry.pack(pady=5)
url_entry.insert(0, load_last_url())  # Load previous URL

mode_var = tk.StringVar()
mode_label = tk.Label(root, text="Open in:", font=('Arial', 11))
mode_label.pack(pady=5)

tab_radio = tk.Radiobutton(root, text="New Tab", variable=mode_var, value="tab")
tab_radio.pack()
window_radio = tk.Radiobutton(root, text="New Window", variable=mode_var, value="window")
window_radio.pack()

open_button = tk.Button(root, text="Open URL", command=open_url, bg="green", fg="white", font=('Arial', 12))
open_button.pack(pady=10)

dark_button = tk.Button(root, text="🌙 Dark Mode", command=toggle_dark_mode, font=('Arial', 10))
dark_button.pack()

root.mainloop()